In [46]:
import requests
import json
import import_ipynb
import login_data as crd
from importnb import Notebook

In [61]:
teamupCalendarEventNotebook=Notebook.load('../teamup/teamupCalendarEvent.ipynb')
tokenExchanger=Notebook.load('../../general/tokenExchanger.ipynb')

In [62]:
class teamupInterface(tokenExchanger.TokenExchanger):
    uniqueCalendarId='kst496bmane3rty9b7'
    
    bidirectionalDictionaryOfTokensAPIvsObject = bidict.bidict(
        {
            #name attribute api, name attribute python object
            'title': 'title',
            'notes': 'text',
            'creation_dt': 'created',
            'update_dt' : 'edited',
            'id': 'st_id',
            'version': 'st_version',
        }
    ) 
    
    # these fields must appear on the left side in the bd attribute
    fieldsNotToWriteToAPI = list(
        {
            'id',
            'version',
        }
    )
    
    def extractFromApi(self):
        response = requests.get('https://api.teamup.com/' + self.uniqueCalendarId + '/events?startDate=2017-08-01&endDate=2021-08-01', headers={
            'Teamup-Token': crd.teamup_api_key
        })
        if not response:
            return 'an error occured requesting the teamup api'
        if response:
            response_content_plain = response.text
            response_content_json = json.loads(response_content_plain)
            events = response_content_json['events'][0]
            parsedEvents = self.convertJSONTokensFromAPIToObjectAsJSON(events)
            return parsedEvents
            # TODO: convert to object
            #return response_content_json['events']
    
    def injectInApi(self, jsonobject):
        reparsedJSON = self.convertJSONTokensFromObjectAsJSONToAPI(jsonobject)
        #TODO: logic to write into the api
        return reparsedJSON

In [63]:
ti = teamupInterface()

In [64]:
parsedEvents=ti.extractFromApi()
print(parsedEvents)

{'series_id': '706262196', 'remote_id': None, 'subcalendar_id': 8090698, 'subcalendar_ids': [8090698], 'all_day': False, 'rrule': 'FREQ=WEEKLY;BYDAY=MO,TU,WE,TH,FR;COUNT=3', 'who': 'hofisi92@gmail.com; hofisi92dev@gmail.com', 'location': 'OTH Regensburg', 'readonly': False, 'tz': 'Europe/Berlin', 'start_dt': '2020-04-30T08:30:00+02:00', 'end_dt': '2020-04-30T09:30:00+02:00', 'ristart_dt': '2020-04-30T06:30:00+00:00', 'rsstart_dt': '2020-04-30T08:30:00+02:00', 'delete_dt': None, 'signup_enabled': True, 'signup_deadline': '2020-04-30T08:30:00+02:00', 'signup_public': False, 'signup_visibility': 'users_with_modify_permission', 'signup_limit': 10, 'comments_enabled': True, 'comments_privacy': 1, 'comments_visibility': 'users_with_modify_permission', 'custom': {}, 'signups': [], 'title': 'Test-Termin', 'text': '<p>Test-Beschreibung</p>', 'created': '2020-04-27T22:46:23+02:00', 'edited': '2020-04-29T13:42:40+02:00', 'st_id': '706262196-rid-1588228200', 'st_version': '5ea968303d158'}


In [65]:
reparsedEvents=ti.injectInApi(parsedEvents)
print(reparsedEvents)

{'title': 'Test-Termin', 'notes': '<p>Test-Beschreibung</p>', 'creation_dt': '2020-04-27T22:46:23+02:00', 'update_dt': '2020-04-29T13:42:40+02:00'}
